In [1]:
"""
SEE: https://github.com/sudoswap/lssvm/blob/main/src/LSSVMPair.sol
Events to query for are:
  1. SwapNFTInPair()
    - emitted by LSSVMPair::swapNFTsForToken (sell order)
  2. SwapNFTOutPair()
    - emitted by LSSVMPair::swapTokenForAnyNFTs (buy order)
  3. FeeUpdate()
    - emitted by LSSVMPair::changeFee
  4. DeltaUpdate()
    - emitted by LSSVMPair::changeDelta, ::swapNFTsForToken, ::swapTokenForAnyNFTs

Pool analysis for a given pair contract.

SHOULD replicate https://github.com/sudoswap/pools-analysis but with historical charts for
pool performance.
"""
# switch to sudo-ape wd first for ape project to work properly
import os
from pathlib import Path

if Path(*Path(os.getcwd()).parts[-2:]) == Path('sudo-ape/notebook'):
    os.chdir('..')

In [3]:
import numpy as np
import pandas as pd
import typing as tp

from ape import accounts, chain, Contract, networks, project

In [4]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [6]:
start_block = 14645816  # pair factory deployment
start_block

14645816

In [10]:
# TODO: change to whatever pool address you want
pool_addr = "0x5caf332dca4e6c9e69d52f320c21e74845353db0"  # xmons
pool = project.LSSVMPair.at(pool_addr)

In [11]:
pool

<LSSVMPair 0x5caF332DcA4e6C9E69d52F320c21e74845353db0>

In [13]:
# curve type
curve = Contract(pool.bondingCurve())
curve

<LinearCurve 0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee>

In [ ]:
# query for historical NFT in swap events
%time query_swaps_in = pool.SwapNFTInPair.query('*', start_block=start_block)
query_swaps_in

INFO: Cache database has not been initialized
